## 1. Einrichtung & Imports

In [80]:
import json
from pathlib import Path
from datetime import datetime

# Data processing
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Alle Bibliotheken erfolgreich importiert")

✅ Alle Bibliotheken erfolgreich importiert


## 2. Konfiguration & Daten Laden

In [81]:
# Configuration
NOTEBOOK_DIR = Path.cwd()
OUTPUT_DIR = NOTEBOOK_DIR / "dax_40_analysis_output"

# Check if output directory exists
if not OUTPUT_DIR.exists():
    raise FileNotFoundError(
        f"Ausgabeverzeichnis nicht gefunden: {OUTPUT_DIR}\n"
        "Bitte führen Sie zuerst dax_40_analysis.ipynb aus, um die Daten zu generieren."
    )

print(f"📁 Ausgabeverzeichnis: {OUTPUT_DIR.absolute()}")
print(f"✅ Verzeichnis existiert mit {len(list(OUTPUT_DIR.glob('*')))} Dateien")

📁 Ausgabeverzeichnis: /Users/andi/Desktop/NAK/code/KnowledgeSolutionCorner/dax_40_analysis_output
✅ Verzeichnis existiert mit 24 Dateien


In [82]:
# Lade alle notwendigen Datendateien
print("📂 Lade Datendateien...\n")

# Zusammenfassungsbericht
with open(OUTPUT_DIR / "summary_report.json", "r") as f:
    summary = json.load(f)
print("✅ summary_report.json geladen")

# DataFrames
df_indicator_freq = pd.read_csv(OUTPUT_DIR / "indicator_frequency.csv")
print(f"✅ indicator_frequency.csv geladen ({len(df_indicator_freq)} Zeilen)")

df_indicator_cov = pd.read_csv(OUTPUT_DIR / "indicator_coverage.csv")
print(f"✅ indicator_coverage.csv geladen ({len(df_indicator_cov)} Zeilen)")

df_dim_stats = pd.read_csv(OUTPUT_DIR / "dimension_statistics.csv")
print(f"✅ dimension_statistics.csv geladen ({len(df_dim_stats)} Zeilen)")

df_dimensions = pd.read_csv(OUTPUT_DIR / "all_dimensions.csv")
print(f"✅ all_dimensions.csv geladen ({len(df_dimensions)} Zeilen)")

df_period_freq = pd.read_csv(OUTPUT_DIR / "period_frequency.csv")
print(f"✅ period_frequency.csv geladen ({len(df_period_freq)} Zeilen)")

print("\n✅ Alle Daten erfolgreich geladen!")
print(f"\n📊 Datenzusammenfassung:")
print(f"   Gesamt Dimensionen: {len(df_dimensions):,}")
print(f"   Eindeutige Indikatoren: {df_dimensions['indicator'].nunique()}")
print(f"   Gesamt Unternehmen: {len(df_dim_stats)}")
print(f"   Berichtszeiträume: {df_dimensions['period'].nunique()}")

📂 Lade Datendateien...

✅ summary_report.json geladen
✅ indicator_frequency.csv geladen (338 Zeilen)
✅ indicator_coverage.csv geladen (338 Zeilen)
✅ dimension_statistics.csv geladen (92 Zeilen)
✅ all_dimensions.csv geladen (12419 Zeilen)
✅ period_frequency.csv geladen (3 Zeilen)

✅ Alle Daten erfolgreich geladen!

📊 Datenzusammenfassung:
   Gesamt Dimensionen: 12,419
   Eindeutige Indikatoren: 338
   Gesamt Unternehmen: 92
   Berichtszeiträume: 3


## 3. Hilfsfunktionen

In [83]:
def categorize_indicator(indicator_name):
    """Categorize indicator by name pattern."""
    if indicator_name in ['sfdr', 'pcaf']:
        return 'Framework Type'
    elif indicator_name.startswith('plainDate'):
        return 'Date Fields'
    elif indicator_name.startswith('plainEnum'):
        return 'Basic Enum'
    elif indicator_name.startswith('extendedEnum'):
        return 'Extended Enum (Yes/No)'
    elif indicator_name.startswith('extendedDecimal'):
        return 'Numeric (Decimal)'
    elif indicator_name.startswith('extendedString'):
        return 'Text Fields'
    else:
        return 'Other'

def categorize_esg(indicator_name):
    """Categorize indicator by ESG theme."""
    indicator_lower = indicator_name.lower()
    
    # Environmental
    env_keywords = ['ghg', 'emission', 'carbon', 'energy', 'renewable', 'water', 'waste', 
                    'biodiversity', 'deforestation', 'climate', 'environmental', 'fossil']
    if any(kw in indicator_lower for kw in env_keywords):
        return 'Environmental'
    
    # Social
    social_keywords = ['labour', 'labor', 'human rights', 'child', 'forced', 'discrimination',
                      'employee', 'accident', 'workplace', 'health', 'safety', 'gender', 
                      'diversity', 'trafficking', 'ilo']
    if any(kw in indicator_lower for kw in social_keywords):
        return 'Social'
    
    # Governance
    gov_keywords = ['corruption', 'bribery', 'governance', 'board', 'whistleblower', 
                   'transparency', 'tax', 'compliance', 'policy', 'legal proceedings']
    if any(kw in indicator_lower for kw in gov_keywords):
        return 'Governance'
    
    # Framework/Meta
    if indicator_name in ['sfdr', 'pcaf'] or 'date' in indicator_lower or 'fiscal' in indicator_lower:
        return 'Framework/Meta'
    
    return 'Other'

print("✅ Hilfsfunktionen definiert")

✅ Hilfsfunktionen definiert


## 4. Zusätzliche Visualisierungen Generieren

In [84]:
# Erstelle zusätzliche Visualisierungen für den umfassenden Bericht
print("🎨 Generiere zusätzliche Visualisierungen für Bericht...\n")

# 1. Indikator-Kategorie-Aufschlüsselung (Kreisdiagramm)
print("[1/6] Erstelle Indikator-Kategorie-Aufschlüsselung...")

# Apply categorization
df_dimensions['category'] = df_dimensions['indicator'].apply(categorize_indicator)
category_counts = df_dimensions['category'].value_counts()

# Create pie chart
fig = px.pie(
    values=category_counts.values,
    names=category_counts.index,
    title='Indikatorverteilung nach Datentyp',
    hole=0.4,  # Donut-Diagramm
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(height=600)
fig.show()
fig.write_html(OUTPUT_DIR / "indicator_category_breakdown.html")
fig.write_image(OUTPUT_DIR / "indicator_category_breakdown.png", width=1200, height=600)
print(f"   ✅ Gespeichert: HTML + PNG")

🎨 Generiere zusätzliche Visualisierungen für Bericht...

[1/6] Erstelle Indikator-Kategorie-Aufschlüsselung...


ValueError: 
Image export using the "kaleido" engine requires the Kaleido package,
which can be installed using pip:

    $ pip install --upgrade kaleido


In [ ]:
# 2. Datenvollständigkeits-Score pro Unternehmen
print("\n[2/6] Erstelle Datenvollständigkeits-Scores...")

# Calculate completeness as percentage of max dimensions
max_dimensions = df_dim_stats['num_dimensions'].max()
df_dim_stats['completeness_score'] = (df_dim_stats['num_dimensions'] / max_dimensions * 100).round(2)

# Top 20 companies with completeness score
top_20_complete = df_dim_stats.nlargest(20, 'num_dimensions').copy()

fig = go.Figure()
fig.add_trace(go.Bar(
    y=top_20_complete['company_name'],
    x=top_20_complete['completeness_score'],
    orientation='h',
    text=top_20_complete['completeness_score'].apply(lambda x: f'{x:.1f}%'),
    textposition='outside',
    marker=dict(
        color=top_20_complete['completeness_score'],
        colorscale='RdYlGn',
        showscale=True,
        colorbar=dict(title="Score (%)")
    ),
    hovertemplate='<b>%{y}</b><br>Score: %{x:.1f}%<br>Dimensions: ' + 
                  top_20_complete['num_dimensions'].astype(str) + '<extra></extra>'
))

fig.update_layout(
    title='Top 20 Unternehmen - Datenvollständigkeits-Score',
    xaxis_title='Vollständigkeits-Score (%)',
    yaxis_title='Unternehmen',
    yaxis={'categoryorder':'total ascending'},
    height=700,
    showlegend=False
)
fig.show()
fig.write_html(OUTPUT_DIR / "data_completeness_scores.html")
fig.write_image(OUTPUT_DIR / "data_completeness_scores.png", width=1200, height=700)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 3. ESG-Kategorieverteilung
print("\n[3/6] Erstelle ESG-Kategorie-Aufschlüsselung...")

# Apply ESG categorization
indicator_esg = df_dimensions.groupby('indicator').size().reset_index(name='count')
indicator_esg['esg_category'] = indicator_esg['indicator'].apply(categorize_esg)
esg_counts = indicator_esg.groupby('esg_category')['count'].sum()

# Create pie chart
fig = px.pie(
    values=esg_counts.values,
    names=esg_counts.index,
    title='ESG-Verteilung - Alle Datenpunkte',
    hole=0.3,
    color_discrete_map={
        'Environmental': '#2ecc71',
        'Social': '#3498db',
        'Governance': '#9b59b6',
        'Framework/Meta': '#95a5a6',
        'Other': '#e74c3c'
    }
)
fig.update_traces(textposition='inside', textinfo='percent+label+value')
fig.update_layout(height=600)
fig.show()
fig.write_html(OUTPUT_DIR / "esg_category_distribution.html")
fig.write_image(OUTPUT_DIR / "esg_category_distribution.png", width=1200, height=600)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 4. Abdeckung wichtiger ESG-Kennzahlen
print("\n[4/6] Erstelle Dashboard für wichtige ESG-Kennzahlen...")

# Definiere wichtige Nachhaltigkeitskennzahlen
key_metrics = {
    'Scope 1 THG-Emissionen': 'extendedDecimalScope1GhgEmissionsInTonnes',
    'Scope 2 THG-Emissionen': 'extendedDecimalScope2GhgEmissionsInTonnes',
    'Scope 3 THG-Emissionen': 'extendedDecimalScope3GhgEmissionsInTonnes',
    'Gesamt THG-Emissionen': 'extendedDecimalScope1And2And3GhgEmissionsInTonnes',
    'Erneuerbare Energie': 'extendedDecimalRenewableEnergyConsumptionInGWh',
    'Wasserverbrauch': 'extendedDecimalWaterConsumptionInCubicMeters',
    'Geschlechtervielfalt Vorstand': 'extendedDecimalBoardGenderDiversityBoardOfDirectorsInPercent',
    'Unfallrate': 'extendedDecimalRateOfAccidents',
    'Kohlenstoffreduktion': 'extendedEnumYesNoCarbonReductionInitiatives',
    'Menschenrechtsrichtlinie': 'extendedEnumYesNoHumanRightsPolicy'
}

# Get coverage for key metrics
key_metrics_coverage = []
for metric_name, metric_id in key_metrics.items():
    coverage_row = df_indicator_cov[df_indicator_cov['indicator'] == metric_id]
    if not coverage_row.empty:
        key_metrics_coverage.append({
            'metric': metric_name,
            'num_companies': coverage_row['num_companies'].iloc[0],
            'coverage_pct': coverage_row['coverage_percentage'].iloc[0]
        })
    else:
        key_metrics_coverage.append({
            'metric': metric_name,
            'num_companies': 0,
            'coverage_pct': 0
        })

df_key_metrics = pd.DataFrame(key_metrics_coverage)

# Create horizontal bar chart
fig = px.bar(
    df_key_metrics.sort_values('coverage_pct', ascending=True),
    y='metric',
    x='coverage_pct',
    orientation='h',
    title='Abdeckung wichtiger ESG-Kennzahlen über Unternehmen',
    labels={'coverage_pct': 'Abdeckung (%)', 'metric': 'ESG-Kennzahl'},
    text='num_companies',
    color='coverage_pct',
    color_continuous_scale='RdYlGn',
    range_color=[0, 100]
)
fig.update_traces(texttemplate='%{text} Unternehmen', textposition='outside')
fig.update_layout(
    height=500,
    showlegend=False,
    xaxis_range=[0, max(50, df_key_metrics['coverage_pct'].max() + 5)]
)
fig.show()
fig.write_html(OUTPUT_DIR / "key_esg_metrics_coverage.html")
fig.write_image(OUTPUT_DIR / "key_esg_metrics_coverage.png", width=1200, height=500)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 5. Unternehmen ohne Daten - Analyse
print("\n[5/6] Analysiere Unternehmen ohne Dimensionen...")

zero_data_companies = df_dim_stats[df_dim_stats['num_dimensions'] == 0].copy()
zero_by_dax = zero_data_companies.groupby('dax_name').size().reset_index(name='num_zero_ids')
zero_by_dax = zero_by_dax.sort_values('num_zero_ids', ascending=False)

fig = px.bar(
    zero_by_dax.head(15),
    x='num_zero_ids',
    y='dax_name',
    orientation='h',
    title='DAX-Unternehmen mit den meisten "Null-Daten" Unternehmens-IDs',
    labels={'num_zero_ids': 'Anzahl IDs ohne Dimensionen', 'dax_name': 'DAX-Unternehmen'},
    text='num_zero_ids'
)
fig.update_traces(textposition='outside', marker_color='#e74c3c')
fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
fig.show()
fig.write_html(OUTPUT_DIR / "companies_with_zero_data.html")
fig.write_image(OUTPUT_DIR / "companies_with_zero_data.png", width=1200, height=600)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 6. Abdeckungsverteilung nach Perzentil
print("\n[6/6] Erstelle Abdeckungs-Perzentil-Analyse...")

# Calculate percentiles
coverage_percentiles = df_indicator_cov['coverage_percentage'].describe(percentiles=[.25, .5, .75, .9, .95, .99])

# Create box plot
fig = go.Figure()
fig.add_trace(go.Box(
    y=df_indicator_cov['coverage_percentage'],
    name='Coverage Distribution',
    boxmean='sd',
    marker_color='#3498db'
))

fig.update_layout(
    title='Verteilung der Indikatorabdeckung über Unternehmen',
    yaxis_title='Abdeckungsprozentsatz (%)',
    showlegend=False,
    height=500
)
fig.show()
fig.write_html(OUTPUT_DIR / "coverage_percentile_boxplot.html")
fig.write_image(OUTPUT_DIR / "coverage_percentile_boxplot.png", width=1200, height=500)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 7. Fehlende Basis-Visualisierungen (aus Original-Analyse)
print("\n[7/11] Erstelle Verteilung der Unternehmens-IDs...")

# Company ID Distribution
company_id_counts = df_dim_stats.groupby('dax_name').size().reset_index(name='num_ids')
company_id_counts = company_id_counts.sort_values('num_ids', ascending=False)

fig = px.bar(
    company_id_counts.head(20),
    x='num_ids',
    y='dax_name',
    orientation='h',
    title='DAX-Unternehmen nach Anzahl der Unternehmens-IDs',
    labels={'num_ids': 'Anzahl Unternehmens-IDs', 'dax_name': 'DAX-Unternehmen'},
    text='num_ids',
    color='num_ids',
    color_continuous_scale='Blues'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=600, yaxis={'categoryorder':'total ascending'})
fig.show()
fig.write_html(OUTPUT_DIR / "company_id_distribution.html")
fig.write_image(OUTPUT_DIR / "company_id_distribution.png", width=1200, height=600)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 8. Top-Unternehmen nach Dimensionen
print("\n[8/11] Erstelle Top-Unternehmen nach Dimensionen...")

top_companies = df_dim_stats.nlargest(20, 'num_dimensions').copy()

fig = px.bar(
    top_companies,
    x='num_dimensions',
    y='company_name',
    orientation='h',
    title='Top 20 Unternehmen nach Anzahl der Datendimensionen',
    labels={'num_dimensions': 'Anzahl Dimensionen', 'company_name': 'Unternehmen'},
    text='num_dimensions',
    color='num_dimensions',
    color_continuous_scale='Viridis'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=700, yaxis={'categoryorder':'total ascending'})
fig.show()
fig.write_html(OUTPUT_DIR / "top_companies_by_dimensions.html")
fig.write_image(OUTPUT_DIR / "top_companies_by_dimensions.png", width=1200, height=700)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 9. Top-Indikatoren nach Häufigkeit
print("\n[9/11] Erstelle Top-Indikatoren...")

top_indicators = df_indicator_freq.head(30)

fig = px.bar(
    top_indicators.sort_values('count', ascending=True),
    x='count',
    y='indicator',
    orientation='h',
    title='Top 30 Indikatoren nach Häufigkeit',
    labels={'count': 'Anzahl Vorkommen', 'indicator': 'Indikator'},
    text='count',
    color='count',
    color_continuous_scale='Oranges'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=800, yaxis={'categoryorder':'total ascending'})
fig.show()
fig.write_html(OUTPUT_DIR / "top_indicators.html")
fig.write_image(OUTPUT_DIR / "top_indicators.png", width=1200, height=800)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 10. Top-Indikatoren nach Abdeckung
print("\n[10/11] Erstelle Top-Indikatoren nach Abdeckung...")

top_coverage = df_indicator_cov.head(30)

fig = px.bar(
    top_coverage.sort_values('coverage_percentage', ascending=True),
    x='coverage_percentage',
    y='indicator',
    orientation='h',
    title='Top 30 Indikatoren nach Unternehmensabdeckung',
    labels={'coverage_percentage': 'Abdeckung (%)', 'indicator': 'Indikator'},
    text='num_companies',
    color='coverage_percentage',
    color_continuous_scale='RdYlGn'
)
fig.update_traces(texttemplate='%{text} Unternehmen', textposition='outside')
fig.update_layout(height=800, yaxis={'categoryorder':'total ascending'})
fig.show()
fig.write_html(OUTPUT_DIR / "top_indicators_by_coverage.html")
fig.write_image(OUTPUT_DIR / "top_indicators_by_coverage.png", width=1200, height=800)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 11. Indikatorabdeckungs-Verteilung (Histogram)
print("\n[11/11] Erstelle Indikatorabdeckungs-Verteilung...")

fig = px.histogram(
    df_indicator_cov,
    x='coverage_percentage',
    nbins=50,
    title='Verteilung der Indikatorabdeckung',
    labels={'coverage_percentage': 'Abdeckung (%)', 'count': 'Anzahl Indikatoren'},
    color_discrete_sequence=['#3498db']
)
fig.update_layout(
    height=500,
    xaxis_title='Abdeckungsprozentsatz (%)',
    yaxis_title='Anzahl Indikatoren',
    showlegend=False
)
fig.show()
fig.write_html(OUTPUT_DIR / "indicator_coverage_distribution.html")
fig.write_image(OUTPUT_DIR / "indicator_coverage_distribution.png", width=1200, height=500)
print(f"   ✅ Gespeichert: HTML + PNG")

In [ ]:
# 12. Dimensionen pro Unternehmen Verteilung
print("\n[12/12] Erstelle Dimensionen pro Unternehmen Verteilung...")

fig = px.histogram(
    df_dim_stats[df_dim_stats['num_dimensions'] > 0],
    x='num_dimensions',
    nbins=40,
    title='Verteilung: Dimensionen pro Unternehmen (nur Unternehmen mit Daten)',
    labels={'num_dimensions': 'Anzahl Dimensionen', 'count': 'Anzahl Unternehmen'},
    color_discrete_sequence=['#2ecc71']
)
fig.update_layout(
    height=500,
    xaxis_title='Anzahl Dimensionen',
    yaxis_title='Anzahl Unternehmen',
    showlegend=False
)
fig.show()
fig.write_html(OUTPUT_DIR / "dimensions_per_company_distribution.html")
fig.write_image(OUTPUT_DIR / "dimensions_per_company_distribution.png", width=1200, height=500)
print(f"   ✅ Gespeichert: HTML + PNG")

print("\n✅ Alle Visualisierungen erfolgreich erstellt!")
print(f"\n📊 Gesamt Visualisierungsdateien: {len(list(OUTPUT_DIR.glob('*.html')))} HTML + {len(list(OUTPUT_DIR.glob('*.png')))} PNG")

## 5. Umfassenden Markdown-Bericht Generieren

In [ ]:
# Generiere umfassenden Markdown-Bericht
print("📝 Generiere umfassenden Markdown-Bericht...\n")

# Calculate additional statistics
companies_with_data = df_dim_stats[df_dim_stats['num_dimensions'] > 0]
companies_without_data = df_dim_stats[df_dim_stats['num_dimensions'] == 0]

# Top indicators for different categories
top_env_indicators = []
top_social_indicators = []
top_gov_indicators = []

for idx, row in df_indicator_freq.head(100).iterrows():
    ind = row['indicator']
    cat = categorize_esg(ind)
    if cat == 'Environmental' and len(top_env_indicators) < 10:
        top_env_indicators.append((ind, row['count']))
    elif cat == 'Social' and len(top_social_indicators) < 10:
        top_social_indicators.append((ind, row['count']))
    elif cat == 'Governance' and len(top_gov_indicators) < 10:
        top_gov_indicators.append((ind, row['count']))

# Hole Berichtszeitraum aus Zusammenfassung
REPORTING_PERIOD_FROM = 2020  # Standard-Fallback
REPORTING_PERIOD_TO = 2025    # Standard-Fallback

# Pre-calculate values that are used in f-strings to avoid issues
thg_metrics = df_key_metrics[df_key_metrics['metric'].str.contains('THG')]
avg_thg_coverage = thg_metrics['coverage_pct'].mean() if len(thg_metrics) > 0 else 0

# Pre-compute DataFrames with renamed columns to avoid f-string escaping issues
category_counts_df = category_counts.to_frame('count').reset_index().rename(columns={'index': 'category'})
esg_counts_df = esg_counts.to_frame('count').reset_index().rename(columns={'index': 'category'})

print("✅ Statistiken für Bericht berechnet")

In [ ]:
# Generiere den vollständigen Markdown-Berichtsinhalt
report_content = f"""# DAX 40 ESG-Datenlandschaft
## Umfassende Analyse der Verfügbarkeit von Nachhaltigkeitsdaten über die Dataland API

---

**Analysedatum:** {datetime.now().strftime('%d. %B %Y')}  
**Berichtsversion:** 1.0  
**Datenquelle:** Dataland API  
**Abdeckung:** DAX 40 Unternehmen  

---

## Zusammenfassung

### 🎯 Haupterkenntnisse

Diese umfassende Analyse untersucht die Verfügbarkeit und Verteilung von Umwelt-, Sozial- und Governance-Daten (ESG) für DAX 40-Unternehmen über die Dataland API. Die Analyse offenbart bedeutende Einblicke in Datenabdeckung, Qualität und Lücken über Deutschlands führenden Aktienindex hinweg.

#### Hauptstatistiken

| Kennzahl | Wert | Anmerkungen |
|----------|------|-------------|
| **Analysierte DAX-Unternehmen** | {summary['data_collection']['dax_40_companies']} | Von 40 insgesamt |
| **Gesamt Unternehmens-IDs** | {summary['data_collection']['total_company_ids']} | Ø {summary['data_collection']['avg_ids_per_company']:.2f} IDs pro Unternehmen |
| **Gesamt Datendimensionen** | {summary['dimensions']['total_dimensions']:,} | Über alle Unternehmen |
| **Eindeutige Indikatoren** | {summary['dimensions']['unique_indicators']} | Unterschiedliche erfasste Datenpunkte |
| **Unternehmen mit Daten** | {summary['coverage']['companies_with_dimensions']} | {summary['coverage']['companies_with_dimensions']/summary['data_collection']['total_company_ids']*100:.1f}% aller IDs |
| **Unternehmen ohne Daten** | {summary['coverage']['companies_without_dimensions']} | {summary['coverage']['companies_without_dimensions']/summary['data_collection']['total_company_ids']*100:.1f}% aller IDs |
| **Ø Dimensionen/Unternehmen** | {summary['dimensions']['avg_dimensions_per_company']:.1f} | Bei Unternehmen mit Daten |

#### 🔴 Kritische Erkenntnisse

1. **Datenverfügbarkeitslücke**: Während {summary['coverage']['companies_with_dimensions']} Unternehmens-IDs ESG-Daten verfügbar haben, haben **{summary['coverage']['companies_without_dimensions']} Unternehmens-IDs ({summary['coverage']['companies_without_dimensions']/summary['data_collection']['total_company_ids']*100:.0f}%) null Dimensionen**, was entweder auf Tochtergesellschaften oder Datenerfassungslücken hindeutet.

2. **Keine universellen Indikatoren**: Es gibt **{summary['coverage']['universal_indicators']} Indikatoren**, die über alle Unternehmen verfügbar sind, was auf fragmentierte Berichtsstandards hinweist.

3. **Begrenzte hohe Abdeckung**: Nur **{summary['coverage']['high_coverage_indicators_80pct']} Indikatoren** erreichen ≥80% Abdeckung über Unternehmen hinweg.

4. **Mehrere IDs pro Unternehmen**: DAX-Unternehmen haben durchschnittlich {summary['data_collection']['avg_ids_per_company']:.2f} Unternehmens-IDs in Dataland, was komplexe Unternehmensstrukturen mit Muttergesellschaften, Tochtergesellschaften und Fonds widerspiegelt.

---

## 1. Methodik

### 1.1 Ansatz zur Datenerhebung

Diese Analyse nutzt die Dataland API, um systematisch die Verfügbarkeit von ESG-Daten über DAX 40-Unternehmen zu erfassen und zu analysieren. Die Methodik besteht aus:

**Schritt 1: Unternehmensidentifizierung**
- Begonnen mit offizieller DAX 40-Unternehmensliste ({summary['data_collection']['dax_40_companies']} Unternehmen im Datensatz enthalten)
- Verwendung der Dataland-Unternehmenssuche-API (`/api/companies/names`) zur Identifizierung aller zugehörigen Unternehmens-IDs
- Jedes DAX-Unternehmen kann mehrere IDs haben, die Muttergesellschaften, Tochtergesellschaften und verbundene Unternehmen repräsentieren

**Schritt 2: Dimensionsextraktion**
- Für jede Unternehmens-ID wurden verfügbare Datendimensionen über `/api/metadata/available-data-dimensions` abgefragt
- Gefiltert nach Berichtszeiträumen: {REPORTING_PERIOD_FROM} - {REPORTING_PERIOD_TO}
- Extrahierte Dimensionsmetadaten einschließlich Indikatortypen, Berichtszeiträumen und Datenpunkt-IDs

**Schritt 3: Datenstrukturierung**
- Normalisiert und kategorisiert alle {summary['dimensions']['total_dimensions']:,} Dimensionen
- Klassifizierte Indikatoren nach:
  - **Datentyp** (Dezimal, Enum, Datum, etc.)
  - **ESG-Kategorie** (Umwelt, Soziales, Governance)
  - **Framework** (SFDR, PCAF, etc.)

**Schritt 4: Analyse**
- Häufigkeitsanalyse: Welche Indikatoren erscheinen am häufigsten?
- Abdeckungsanalyse: Welche Indikatoren sind über Unternehmen hinweg verfügbar?
- Lückenanalyse: Welchen Unternehmen fehlen Daten?
- Querschnittsanalyse: Muster und Korrelationen

### 1.2 Umfang und Einschränkungen

**Berichtszeitraum:** {REPORTING_PERIOD_FROM} - {REPORTING_PERIOD_TO}

**Einschränkungen:**
- Analyse fokussiert auf *Verfügbarkeit* von Daten, nicht auf Qualität oder Genauigkeit
- Mehrere Unternehmens-IDs pro DAX-Entität können Muster aufblähen oder verschleiern
- Null-Dimensions-IDs können legitime Tochtergesellschaften ohne separate Berichterstattung darstellen
- API-Zugriff beschränkt auf öffentlich verfügbare/erlaubte Daten

---

## 2. Analyse der Unternehmensabdeckung

### 2.1 Verteilung der Unternehmens-IDs

Die {summary['data_collection']['dax_40_companies']} analysierten DAX-Unternehmen werden **{summary['data_collection']['total_company_ids']} eindeutigen Unternehmens-IDs** im Dataland-System zugeordnet, durchschnittlich **{summary['data_collection']['avg_ids_per_company']:.2f} IDs pro DAX-Unternehmen**.

Diese Vielzahl spiegelt wider:
- Muttergesellschaft + Tochtergesellschaftsbeziehungen
- Investmentfonds und Zweckgesellschaften
- Internationale Niederlassungen und regionale Einheiten
- Separate juristische Personen innerhalb von Unternehmensgruppen

![Verteilung der Unternehmens-IDs](dax_40_analysis_output/company_id_distribution.png)

### 2.2 Datenverfügbarkeit nach Unternehmen

#### Top 20 Unternehmen nach Datendimensionen

{df_dim_stats.nlargest(20, 'num_dimensions')[['dax_name', 'company_name', 'num_dimensions']].to_markdown(index=False)}

**Beobachtungen:**
- Top-Unternehmen hat **{df_dim_stats['num_dimensions'].max()} Dimensionen**
- Top-Performer gruppieren sich um **350-450 Dimensionen**
- Starke Vertretung aus Industrie-, Automobil- und Energiesektoren

![Top-Unternehmen nach Dimensionen](dax_40_analysis_output/top_companies_by_dimensions.png)

#### Datenvollständigkeits-Scores

Vollständigkeit wird als Prozentsatz der maximal verfügbaren Dimensionen ({df_dim_stats['num_dimensions'].max()}) berechnet:

| Vollständigkeitsstufe | Anzahl Unternehmen | Prozentsatz |
|-----------------------|--------------------|-------------|
| 90-100% (Hervorragend) | {len(companies_with_data[companies_with_data['num_dimensions'] >= 0.9 * df_dim_stats['num_dimensions'].max()])} | {len(companies_with_data[companies_with_data['num_dimensions'] >= 0.9 * df_dim_stats['num_dimensions'].max()])/len(df_dim_stats)*100:.1f}% |
| 70-89% (Gut) | {len(companies_with_data[(companies_with_data['num_dimensions'] >= 0.7 * df_dim_stats['num_dimensions'].max()) & (companies_with_data['num_dimensions'] < 0.9 * df_dim_stats['num_dimensions'].max())])} | {len(companies_with_data[(companies_with_data['num_dimensions'] >= 0.7 * df_dim_stats['num_dimensions'].max()) & (companies_with_data['num_dimensions'] < 0.9 * df_dim_stats['num_dimensions'].max())])/len(df_dim_stats)*100:.1f}% |
| 50-69% (Ausreichend) | {len(companies_with_data[(companies_with_data['num_dimensions'] >= 0.5 * df_dim_stats['num_dimensions'].max()) & (companies_with_data['num_dimensions'] < 0.7 * df_dim_stats['num_dimensions'].max())])} | {len(companies_with_data[(companies_with_data['num_dimensions'] >= 0.5 * df_dim_stats['num_dimensions'].max()) & (companies_with_data['num_dimensions'] < 0.7 * df_dim_stats['num_dimensions'].max())])/len(df_dim_stats)*100:.1f}% |
| 1-49% (Begrenzt) | {len(companies_with_data[(companies_with_data['num_dimensions'] > 0) & (companies_with_data['num_dimensions'] < 0.5 * df_dim_stats['num_dimensions'].max())])} | {len(companies_with_data[(companies_with_data['num_dimensions'] > 0) & (companies_with_data['num_dimensions'] < 0.5 * df_dim_stats['num_dimensions'].max())])/len(df_dim_stats)*100:.1f}% |
| 0% (Keine Daten) | {len(companies_without_data)} | {len(companies_without_data)/len(df_dim_stats)*100:.1f}% |

![Datenvollständigkeits-Scores](dax_40_analysis_output/data_completeness_scores.png)

### 2.3 Unternehmen ohne Dimensionen

**{len(companies_without_data)} Unternehmens-IDs ({len(companies_without_data)/len(df_dim_stats)*100:.1f}%)** haben null Datendimensionen. Dies sind hauptsächlich:
- Tochtergesellschaften und Zweckgesellschaften
- Investmentfonds und Finanzinstrumente
- Regionale Niederlassungen ohne separate Berichterstattung
- Entitäten möglicherweise außerhalb des aktuellen Datenerfassungsumfangs

**Top DAX-Unternehmen mit den meisten Null-Daten-IDs:**

{zero_by_dax.head(10).to_markdown(index=False)}

![Unternehmen ohne Daten](dax_40_analysis_output/companies_with_zero_data.png)

---

## 3. Indikatoranalyse

### 3.1 Verteilung der Indikatorhäufigkeit

Von den **{summary['dimensions']['unique_indicators']} eindeutigen Indikatoren** variiert die Häufigkeit dramatisch:

#### Top 30 häufigste Indikatoren

{df_indicator_freq.head(30).to_markdown(index=False)}

**Hauptbeobachtungen:**
- Top-Indikatoren erscheinen häufig über Dimensionen hinweg
- Framework-Identifikatoren (SFDR, PCAF) und Metadatenfelder (Daten, Geschäftsjahr) dominieren
- Ja/Nein-Policy-Indikatoren sind hochfrequent
- Langer Schwanz: viele Indikatoren erscheinen sehr selten

![Top-Indikatoren](dax_40_analysis_output/top_indicators.png)

### 3.2 Aufschlüsselung nach Indikatortyp

Indikatoren kategorisiert nach Datentyp:

{category_counts_df.to_markdown(index=False)}

**Analyse:**
- **Extended Enum (Ja/Nein)** Indikatoren dominieren, was policy-orientierte ESG-Berichterstattung widerspiegelt
- **Numeric (Dezimal)** Indikatoren liefern quantitative ESG-Metriken
- Framework-Typ-Indikatoren (SFDR, PCAF) dienen als kategorische Marker

![Indikator-Kategorie-Aufschlüsselung](dax_40_analysis_output/indicator_category_breakdown.png)

### 3.3 ESG-Kategorieverteilung

Kategorisierung der Indikatoren nach ESG-Thema:

{esg_counts_df.to_markdown(index=False)}

![ESG-Kategorieverteilung](dax_40_analysis_output/esg_category_distribution.png)

### 3.4 Abdeckungsanalyse

#### Indikatoren mit hoher Abdeckung

Die **maximale Abdeckung** beträgt ungefähr **{df_indicator_cov['coverage_percentage'].max():.1f}%** ({df_indicator_cov.iloc[0]['num_companies']} von {summary['data_collection']['total_company_ids']} Unternehmen).

#### Top 20 Indikatoren nach Abdeckung

{df_indicator_cov.head(20).to_markdown(index=False)}

**Beobachtungen:**
- Selbst die häufigsten Indikatoren erreichen nur ~{df_indicator_cov['coverage_percentage'].max():.0f}% Abdeckung
- Abdeckung fällt schnell über Top-Indikatoren hinaus ab
- Deutet entweder auf:
  - Heterogene Berichtspraktiken
  - Unterschiedliche Framework-Adoptionsraten
  - Unvollständige Datenerfassung

![Verteilung der Indikatorabdeckung](dax_40_analysis_output/indicator_coverage_distribution.png)

![Top-Indikatoren nach Abdeckung](dax_40_analysis_output/top_indicators_by_coverage.png)

---

## 4. Querschnittsanalyse

### 4.1 Indikator-Unternehmens-Matrix

Die Heatmap-Visualisierung zeigt Clustering-Muster:
- Bestimmte Unternehmen berichten umfassend über viele Indikatoren
- Einige Indikatorgruppen tendieren dazu, zusammen zu erscheinen (Framework-Konformität)
- Klare Lücken, wo Unternehmen spezifische Indikatorkategorien fehlen

![Indikator-Unternehmens-Heatmap](dax_40_analysis_output/indicator_company_heatmap.png)

### 4.2 Dimensionsverteilung

Statistische Verteilung der Dimensionen pro Unternehmen:

{df_dim_stats['num_dimensions'].describe().to_frame('Dimensionen pro Unternehmen').to_markdown()}

![Verteilung der Dimensionen pro Unternehmen](dax_40_analysis_output/dimensions_per_company_distribution.png)

---

## 5. Vertiefung: Wichtige ESG-Kennzahlen

### 5.1 Kritische Nachhaltigkeitsindikatoren

Analyse der Verfügbarkeit der wichtigsten ESG-Kennzahlen:

{df_key_metrics.to_markdown(index=False)}

**Analyse:**
- **THG-Emissionsdaten** (Scope 1, 2, 3) haben ähnliche Abdeckung (~{avg_thg_coverage:.1f}%)
- **Energieverbrauchs**-Metriken mäßig vertreten
- **Policy-Indikatoren** (Menschenrechte, Kohlenstoffreduktion) zeigen variierende Abdeckung
- Verfügbarkeit von **Vorstandsvielfalt** und **Unfallrate**-Daten variiert

![Abdeckung wichtiger ESG-Kennzahlen](dax_40_analysis_output/key_esg_metrics_coverage.png)

### 5.2 Umwelt-Kennzahlen - Top 10

{pd.DataFrame(top_env_indicators, columns=['Indikator', 'Anzahl']).to_markdown(index=False) if top_env_indicators else '*Keine Umwelt-Indikatoren in Top-Daten*'}

### 5.3 Sozial-Kennzahlen - Top 10

{pd.DataFrame(top_social_indicators, columns=['Indikator', 'Anzahl']).to_markdown(index=False) if top_social_indicators else '*Keine Sozial-Indikatoren in Top-Daten*'}

### 5.4 Governance-Kennzahlen - Top 10

{pd.DataFrame(top_gov_indicators, columns=['Indikator', 'Anzahl']).to_markdown(index=False) if top_gov_indicators else '*Keine Governance-Indikatoren in Top-Daten*'}

---

## 6. Datenqualität & Lücken

### 6.1 Zusammenfassung der Datenabdeckung

| Kategorie | Status | Anzahl | Prozentsatz |
|-----------|--------|--------|-------------|
| ✅ Unternehmen mit vollständigen Daten (>400 dims) | Gut | {len(companies_with_data[companies_with_data['num_dimensions'] > 400])} | {len(companies_with_data[companies_with_data['num_dimensions'] > 400])/len(df_dim_stats)*100:.1f}% |
| 🟡 Unternehmen mit teilweisen Daten (1-400 dims) | Ausreichend | {len(companies_with_data[companies_with_data['num_dimensions'] <= 400])} | {len(companies_with_data[companies_with_data['num_dimensions'] <= 400])/len(df_dim_stats)*100:.1f}% |
| 🔴 Unternehmen ohne Daten (0 dims) | Schlecht | {len(companies_without_data)} | {len(companies_without_data)/len(df_dim_stats)*100:.1f}% |

### 6.2 Fehlende Datenmuster

**Primäre Lücke:** Die Mehrheit ({len(companies_without_data)/len(df_dim_stats)*100:.1f}%) der Unternehmens-IDs haben null Datendimensionen.

**Mögliche Erklärungen:**
1. **Tochtergesellschaften** ohne unabhängige ESG-Berichterstattung
2. **Datenerfassung** noch nicht auf alle Entitäten ausgedehnt
3. **Framework-Nicht-Teilnahme** - Unternehmen berichten nicht unter SFDR/PCAF
4. **Kürzliche Ergänzungen** zur Dataland-Datenbank ohne Datenaufnahme

### 6.3 Indikatorlücken

**Indikatoren mit niedriger Abdeckung** (unteres Perzentil):

{df_indicator_cov.tail(20).to_markdown(index=False)}

Diese seltenen Indikatoren können darstellen:
- Unternehmensspezifische oder branchenspezifische Metriken
- Neu eingeführte Berichtsanforderungen
- Optionale/fortgeschrittene ESG-Offenlegungen
- Laufende Datenerfassung

---

## 7. Empfehlungen

### 7.1 Für Datennutzer

1. **Fokus auf abgedeckte Unternehmen**: Priorisieren Sie die Analyse der {len(companies_with_data)} Unternehmen mit substanziellen Daten
2. **Framework-Ausrichtung**: SFDR- und PCAF-Frameworks zeigen stärkste Abdeckung
3. **Indikatorauswahl**: Verwenden Sie Top-Abdeckungs-Indikatoren für Unternehmensvergleiche
4. **Lücken beachten**: Seien Sie sich bewusst, dass {len(companies_without_data)/len(df_dim_stats)*100:.0f}% der IDs keine Daten haben; validieren Sie Entitätsrelevanz

### 7.2 Für Datenanbieter

1. **Tochtergesellschafts-Abdeckung erweitern**: Klären Sie, welche Entitäten unabhängige Berichterstattung haben sollten
2. **Frameworks standardisieren**: Arbeiten Sie auf universelle Adoption von SFDR/PCAF hin
3. **Indikatorlücken schließen**: Zielen Sie auf Metriken mit niedriger Abdeckung aber hoher Wichtigkeit (z.B. Vorstandsvielfalt, Wasserverbrauch)
4. **Datenqualitätsflags**: Unterscheiden Sie zwischen "nicht anwendbar" vs. "nicht berichtet" vs. "in Bearbeitung"

### 7.3 Prioritätsziele

**Unternehmen, die Datenerfassungsaufmerksamkeit benötigen:**
- Alle {len(companies_without_data)} Null-Dimensions-Entitäten (Relevanz validieren)
- Unternehmen unter 50% Vollständigkeitsschwelle

**Indikatoren, die breitere Adoption benötigen:**
- Geschlechtervielfalt im Vorstand (Governance)
- Wasserverbrauch (Umwelt)
- Unfallrate & Arbeitsplatzsicherheit (Soziales)
- Scope 3 THG-Emissionen (Umwelt)

---

## 8. Anhänge

### Anhang A: Top 100 Indikatoren nach Häufigkeit

{df_indicator_freq.head(100).to_markdown(index=False)}

### Anhang B: Unternehmensübersicht

Vollständige Auflistung aller Unternehmen mit Dimensionszahlen:

{df_dim_stats.sort_values('num_dimensions', ascending=False).to_markdown(index=False)}

### Anhang C: Interaktive Visualisierungen

Alle Visualisierungen sind im Ausgabeverzeichnis verfügbar:

- Verteilung der Unternehmens-IDs (PNG)
- Top-Unternehmen nach Dimensionen (PNG)
- Top-Indikatoren (PNG)
- Top-Indikatoren nach Abdeckung (PNG)
- Verteilung der Indikatorabdeckung (PNG)
- Indikator-Unternehmens-Heatmap (PNG)
- Indikator-Kategorie-Aufschlüsselung (PNG)
- Datenvollständigkeits-Scores (PNG)
- ESG-Kategorieverteilung (PNG)
- Abdeckung wichtiger ESG-Kennzahlen (PNG)
- Unternehmen ohne Daten (PNG)
- Abdeckungs-Perzentil-Boxplot (PNG)

*Interaktive HTML-Versionen sind ebenfalls verfügbar für detaillierte Exploration.*

### Anhang D: Datenwörterbuch

**Schlüsselbegriffe:**

- **Unternehmens-ID**: Eindeutige Kennung im Dataland-System für eine juristische Person
- **Dimension**: Ein einzelner Datenpunkt (Indikator + Unternehmen + Zeitraum)
- **Indikator**: Eine spezifische ESG-Kennzahl oder Datenfeldtyp
- **Abdeckung**: Prozentsatz der Unternehmen, die einen bestimmten Indikator berichten
- **Framework**: Standardisiertes Berichtsschema (SFDR, PCAF, etc.)

**ESG-Kategorien:**
- **Umwelt**: Klima, Energie, Wasser, Abfall, Biodiversität
- **Soziales**: Arbeitsrechte, Menschenrechte, Gesundheit & Sicherheit, Vielfalt
- **Governance**: Anti-Korruption, Vorstandsstruktur, Transparenz, Compliance

---

## Fazit

Diese umfassende Analyse der Verfügbarkeit von DAX 40 ESG-Daten über die Dataland API offenbart eine Landschaft, die charakterisiert ist durch:

1. **Substanzielle Daten für Kernunternehmen**: {len(companies_with_data)} Entitäten liefern ESG-Daten
2. **Fragmentierte Abdeckung**: Keine universellen Indikatoren; maximale Abdeckung ~{df_indicator_cov['coverage_percentage'].max():.0f}%
3. **Framework-Konzentration**: SFDR- und PCAF-Frameworks dominant
4. **Signifikante Lücken**: {len(companies_without_data)/len(df_dim_stats)*100:.0f}% der Unternehmens-IDs fehlen Daten (hauptsächlich Tochtergesellschaften)
5. **Policy-lastige Berichterstattung**: Ja/Nein-Policy-Indikatoren überwiegen quantitative Metriken

**Der Weg nach vorn** erfordert Standardisierung, breitere Framework-Adoption und Klärung der Berichtsverantwortlichkeiten über Unternehmensstrukturen hinweg.

---

**Bericht erstellt:** {datetime.now().strftime('%d.%m.%Y %H:%M:%S')}  
**Quell-Notebook:** `dax_40_report_generator.ipynb`  
**Datenverzeichnis:** `dax_40_analysis_output/`  

---
"""

# Speichere den Bericht
report_path = OUTPUT_DIR / "DAX_40_ESG_Data_Landscape_Report.md"
with open(report_path, "w", encoding="utf-8") as f:
    f.write(report_content)

print(f"✅ Umfassender Markdown-Bericht generiert!")
print(f"📄 Bericht gespeichert unter: {report_path}")
print(f"📊 Berichtslänge: {len(report_content):,} Zeichen")

print(f"📑 Berichtssektionen: Zusammenfassung, Methodik, Unternehmensabdeckung, Indikatoranalyse, Querschnittsanalyse, ESG-Vertiefung, Datenqualität & Lücken, Empfehlungen, Anhänge")print(f"\n🎉 Berichtsgenerierung abgeschlossen!")

## 6. Zusammenfassung

In [ ]:
print("=" * 80)
print("📊 BERICHTSGENERIERUNG - ZUSAMMENFASSUNG")
print("=" * 80)
print(f"\n✅ Umfassender DAX 40 ESG-Datenlandschaftsbericht erfolgreich generiert")
print(f"\n📁 Ausgabeverzeichnis: {OUTPUT_DIR.absolute()}")
print(f"\n📄 Berichtsdatei: {OUTPUT_DIR / 'DAX_40_ESG_Data_Landscape_Report.md'}")
print(f"\n📊 Erstellte Visualisierungen:")
viz_files = list(OUTPUT_DIR.glob('*.html')) + list(OUTPUT_DIR.glob('*.png'))
for i, viz_file in enumerate(sorted(viz_files), 1):
    print(f"   {i:2d}. {viz_file.name}")
print(f"\n✨ Gesamt Dateien im Ausgabeverzeichnis: {len(list(OUTPUT_DIR.glob('*')))}")
print(f"\n🎉 Alles erledigt! Öffnen Sie den Markdown-Bericht, um die umfassende Analyse zu sehen.")